## Relations

Compute irreducible relations for neuron systems found to have φ>0.

Isaac David, February 2022.

In [51]:
from temporal_emergence import CoarseGrainer, PhiCalculator, Helpers
from RepertoirePlotter import RepertoirePlotter
import numpy as np
import pyphi
from os import listdir
import pandas as pd
import re

pyphi.config.VALIDATE_CONDITIONAL_INDEPENDENCE = False
pyphi.config.REPERTOIRE_DISTANCE = 'AID'
pyphi.config.CES_DISTANCE = 'SUM_SMALL_PHI'

def get_mechanism_phis(relation_mechanisms, sia_mechanisms):
    '''Get core small phis for the mechanisms involved in a given relation.

    Parameters:
        relation_mechanisms (Generator[tuple]): indices of mechanisms in relata.
        sia_mechanisms (pyphi.models.subsystem.CauseEffectStructure): ces object
           coming from a system irreducibility analysis.

    Returns:
        list[float]: list of small phi values (one per relata), in the same
                     order as the relata.
    '''
    mech_phis = []
    for m in relation_mechanisms: # relation.relata.mechanisms:
        for n in sia_mechanisms: # sia.ces:
            if m == n.mechanism:
                mech_phis.append(n.phi)
    return mech_phis


# get pairs of connected neurons
bidirectionally = []
with open("bidirectionally.txt", "r") as f:
    for line in f:
        bidirectionally.append(
            "micro_" + line.replace(' ', '_').replace('\n', '')
        )

pattern = '(' + '|'.join(bidirectionally) + ')'

TPMs_connected = filter(lambda path: re.match(pattern, path) is not None and 'csv' in path and '200_bin_0.0178625_skip_2' in path,
                        listdir('./results/macro_micros_fixed'))
TPMs_unconnected = filter(lambda path: re.match(pattern, path) is None and 'csv' in path and '200_bin_0.0178625_skip_2' in path,
                          listdir('./results/macro_micros_fixed'))

TPMs = [[TPM, True] for TPM in list(TPMs_connected)] + \
    [[TPM, False] for TPM in list(TPMs_unconnected)]

# declare empty dict structure, this will be used to store results in
# a dataframe
df = {
    'neurons': [],
    'connected': [],
    'state': [],
    'phi': [],
    'mech_phis': [],
    'big_Phi': [],
    'relata': [],
    'order': [],
    'type': [],
    'degree': [],
    'completeness': [],
    'overlap': []
}

#                               OR               AND            3-STATE             IDENTITY
element_coarse_grainings = [[[0], [1,2,3]], [[0,1,2],[3]], [[0], [1,2], [3]], [[0], [1], [2], [3]]]
states, num_states_l = CoarseGrainer.get_state_maps(element_coarse_grainings)

# iterate over neuron pairs and compute relations
count = 0
for TPM in TPMss:
    print(str(count) + '/' + str(len(TPMs)))
    count=count+1
    TPM_current = np.loadtxt('./results/macro_micros_fixed/' + TPM[0])

    # TODO: loop through states and num_states (different grainings)
    TPM_current_coarse_grained = CoarseGrainer.coarse_grain_nonbinary_TPM(
        TPM_current, states[0], num_states_l[0]
    )
    network = pyphi.Network(TPM_current_coarse_grained)
    current_network_states = [(0,0), (0,1), (1,0), (1,1)]

    for state in current_network_states:
        subsystem = pyphi.Subsystem(network, state)
        sia = pyphi.compute.sia(subsystem)
        relations = pyphi.relations.relations(subsystem, sia.ces)

        for relation in relations:
            neurons=TPM[0][6:].replace('_', ' ', 1) # 'micro_1_2_ ...' -> '1 2_ ...'
            neurons=neurons[:neurons.find('_')]     # '1 2_ ...' -> '1 2'
            df['neurons'].append(neurons.split(' '))
            df['connected'].append(TPM[1])
            df['state'].append(state)
            df['phi'].append(relation.phi)
            df['mech_phis'].append(
                get_mechanism_phis(relation.relata.mechanisms, sia.ces)
            )
            df['big_Phi'].append(sia.phi)
            df['relata'].append(str(relation.relata))
            df['order'].append(len(set(relation.mechanisms)))
            df['type'].append(
                list(np.sort([x.name for x in relation.relata.directions]))
            )
            # relation.degree
            df['degree'].append(len([x for x in relation.relata.purviews]))
            df['completeness'].append(
                2 * len(set(relation.mechanisms)) == \
                len([x for x in relation.relata.purviews])
            )
            df['overlap'].append(len(set([x for x in relation.relata.purviews])))

df2 = pd.DataFrame(df)
df2.to_csv('relations.csv')
print(df2)

52


NameError: name 'TPMss' is not defined